In [1]:
!pip install pandas numpy scikit-surprise
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
from pandas import DataFrame



# Define movie names mapping
movie_names = {
    '101': 'Inception',
    '102': 'The Dark Knight',
    '103': 'Interstellar',
    '104': 'Pulp Fiction',
    '105': 'The Shawshank Redemption',
    '106': 'Forrest Gump',
    '107': 'Fight Club',
    '108': 'The Matrix',
    '109': 'The Godfather',
    '110': 'Schindler\'s List'
}

# Function to take user input
def take_user_input():
    user_input = {}
    num_users = int(input("Enter the number of users: "))
    for i in range(1, num_users + 1):
        user_id = str(i)
        print(f"For user {user_id}:")
        num_ratings = int(input("Enter the number of ratings: "))
        ratings = []
        for j in range(num_ratings):
            item_id = input(f"Enter movie ID for rating {j+1}: ")
            rating = float(input(f"Enter rating for movie '{movie_names[item_id]}': "))
            while rating >= 5:  # Ensure rating is below 5
                print("Rating should be below 5.")
                rating = float(input(f"Enter rating for movie '{movie_names[item_id]}': "))
            ratings.append((item_id, rating))
        user_input[user_id] = ratings
    return user_input

# Take user input
user_input = take_user_input()

# Create a DataFrame from the user input
# Create a DataFrame from the user input
user_df_new = pd.DataFrame(columns=["user_id", "item_id", "rating"])
for user_id, ratings in user_input.items():
    for item_id, rating in ratings:
        print(type(user_df_new))  # Print the type of user_df_new
        # user_df_new = user_df_new.append({'user_id': user_id, 'item_id': item_id, 'rating': rating}, ignore_index=True)
        user_df_new = pd.concat([user_df_new, pd.DataFrame({'user_id': [user_id], 'item_id': [item_id], 'rating': [rating]})], ignore_index=True)


# Define the reader
reader = Reader(rating_scale=(1, 5))

# Load the data into Surprise's dataset format
# data = Dataset.load_from_df(user_df[["user_id", "item_id", "rating"]], reader)
data = Dataset.load_from_df(user_df_new[["user_id", "item_id", "rating"]], reader)


# Split the dataset into train and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Define the collaborative filtering model (we'll use KNNBasic)
sim_options = {
    'name': 'cosine',
    'user_based': False  # item-based collaborative filtering
}

# Initialize the model
model = KNNBasic(sim_options=sim_options)

# Train the model
model.fit(trainset)

# Make predictions
predictions = model.test(testset)

# Evaluate the model
accuracy = rmse(predictions)
print("RMSE:", accuracy)

# Generate recommendations for each user
for user_id in user_input.keys():
    unrated_items = [item_id for item_id in movie_names.keys() if item_id not in [rating[0] for rating in user_input[user_id]]]

    predictions = {}
    for item_id in unrated_items:
        predicted_rating = model.predict(user_id, item_id).est
        predictions[item_id] = predicted_rating

    sorted_predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
    top_recommendations = sorted_predictions[:2]  # Get top 2 recommendations

    print("Top 2 recommendations for user", user_id, ":")
    for item_id, predicted_rating in top_recommendations:
        print("Movie:", movie_names[item_id], "- Predicted Rating:", predicted_rating)
    print()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162999 sha256=051275ced544d73959d686a646441ea557be7d21a1437d21055558c777fb64ac
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise
Enter the number of users: 1
For user 1:
Enter the number of ratings: 1
Enter movie ID for rating 1: 102
Enter rating for movie 'The Dark Knight': 4
<class 'pandas.core.frame.DataFrame'>
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0000
RMSE: 1.0
Top 2 recommendations for user 1 :
Movie: Inception - Predicted Rating: 5
Movie: Interstellar - Predicted Rating: 5



/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
